In [1]:
Threads.nthreads() = 12

## Importing Packages

In [2]:
"""
Importing (using/include) packages and files needed for the code to run

Topo.jl --- Topography Setup

Note: that we import pyplot last to avoid a name conflict with grid
"""

using SparseArrays
using LinearAlgebra
using IterativeSolvers
using WriteVTK
using Printf
using Statistics
using Dates
using HDF5
include("Grid.jl")
include("Markers.jl")
include("Stokes.jl")
include("Temperature.jl")
include("GridOperations.jl")
# Note: that we import pyplot last to avoid a name conflict with grid
using PyPlot
include("Visualization.jl")

plots (generic function with 1 method)

# Topography Setup

In [3]:
function initial_ice_depth(x::Float64,ice_thickness::Float64,wavelength::Float64,amplitude::Float64,initial_surface_depth::Float64)
    return ice_thickness + initial_surface_depth + amplitude*sin( 2*pi/wavelength*x )
end

function ice_viscosity(T::Float64)
    """
    Arguments:
    T::Float64 --- A temperature value in (Kelvin) of float64

    Returns:
    ice_vis -- The ice viscosity in (Pa*s) that is dependent on temperature argument value

    Info:
    The equation used is a Newtonian formulation for temperature-dependent ice viscosity from Nimmo(2004)
    """
    Q = 40000.0 # Activation Enegry (J/mol)
    R_cont = 8.314 # Gas Constant (J/mol*K)
    ice_vis = (1e15)*exp((Q*(273.0-T))/(R_cont*(273.0*T)))
    upperlimit = 1e25
    lowerlimit = 1e12
    if ice_vis < lowerlimit
        ice_vis = lowerlimit
    elseif ice_vis > upperlimit
        ice_vis = upperlimit
    else 
        ice_vis = ice_vis
    end 
    return ice_vis
end

ice_viscosity (generic function with 1 method)

# Material Setup

In [4]:
struct Materials
    """
    Building composite types with a list of fields (or attributes) which are names that will be used as variables to store data for
    objects of the new types.

    Fields:
    alpha::Vector{Float64} -- A vector(1-D Array) that holds values of Thermal expansion in (1/K)
    rho0::Vector{Float64} -- A vector (1-D Array) that holds values of  Density in (kg/m^3)
    Hr::Vector{Float64} -- A vector(1-D Array) that holds values of Radiogenic heat production in (W/m^3)
    Cp::Vector{Float64} -- A vector(1-D Array) that holds values of Heat capacity in (J/kg*K)
    kThermal::Vector{Float64} -- A vector(1-D Array) that holds values of Thermal conductivity in (W/m*K)
    eta::Vector{Float64} -- A vector(1-D Array) that holds values of Viscosity in (Pa*s = kg/m*s)

    Info:
    Each vector hold 3 elements which correspond to layer
    # 1 - subsurface global ocean
    # 2 - icy shell
    # 3 - sticky air
    """
    alpha::Vector{Float64} # Thermal expansion (1/K)
    rho0::Vector{Float64} # Density (kg/m^3)
    Hr::Vector{Float64} # Radiogenic heat production (W/m^3)
    Cp::Vector{Float64} # Heat capacity (J/kg*K)
    kThermal::Vector{Float64} # Thermal conductivity (W/m*K)
    eta::Vector{Float64} # Viscosity (Pa*s)
    function Materials()
        new([0.0,0.0,0.0],[1000.0,920.0,1.0],[0.0,0.0,0.0],[4180.0,2100.0,1.0e6],[0.5610,2.1,0.024],[1e12,1e15,1e17])
    end    
end

function update_marker_prop!(markers::Markers,materials::Materials)
    eta = markers.scalarFields["eta"]
    rho = markers.scalarFields["rho"]
    T = markers.scalarFields["T"]
    mmat = markers.integers[markers.integerFields["material"],:]
    for i in 1:markers.nmark
        markers.scalars[rho,i] = materials.rho0[mmat[i]]
        if mmat[i] == 2
            markers.scalars[eta,i] = ice_viscosity(markers.scalars[T,i])
        end
    end
end

function update_marker_temp!(markers::Markers,materials::Materials)
    T = markers.scalarFields["T"]
    mmat = markers.integers[markers.integerFields["material"],:]
    for i in 1:markers.nmark
        if mmat[i] == 1 
            markers.scalars[T,i] = 273.0
        elseif mmat[i] == 3 
            markers.scalars[T,i] = 100.0
        end 
    end 
end 

function initial_conditions!(markers::Markers,materials::Materials,options::Dict)
    material = markers.integerFields["material"]
    T = markers.scalarFields["T"]
    eta = markers.scalarFields["eta"]
    alpha = markers.scalarFields["alpha"]
    Cp = markers.scalarFields["Cp"]
    Hr = markers.scalarFields["Hr"]
    kThermal = markers.scalarFields["kThermal"]
    for i in 1:markers.nmark
        mx = markers.x[1,i]
        my = markers.x[2,i]
        hice = initial_ice_depth(mx,options["ice thickness"],options["wavelength"],options["amplitude"],options["surface depth"])
        hsurf = options["surface depth"]
        if my > hice
            # subsurface global ocean
            markers.integers[material,i] = 1
            markers.scalars[T,i] = 273.0
            markers.scalars[eta,i] = materials.eta[1]
            markers.scalars[alpha,i] = materials.alpha[1]        
            markers.scalars[Cp,i] = materials.Cp[1]
            markers.scalars[Hr,i] = materials.Hr[1] 
            markers.scalars[kThermal,i] = materials.kThermal[1]
        elseif my > hsurf
            # icy shell
            markers.integers[material,i] = 2
            markers.scalars[T,i] = 100.0+((273.0-100.0)/(hice-hsurf))*(my-hsurf)
            # markers.scalars[eta,i] = eta_i[i]
            markers.scalars[alpha,i] = materials.alpha[2]
            markers.scalars[Cp,i] = materials.Cp[2]
            markers.scalars[Hr,i] = materials.Hr[2]
            markers.scalars[kThermal,i] = materials.kThermal[2]
        else
            # sticky air
            markers.integers[material,i] = 3
            markers.scalars[T,i] = 100.0            
            markers.scalars[eta,i] = materials.eta[3]
            markers.scalars[alpha,i] = materials.alpha[3]  
            markers.scalars[Cp,i] = materials.Cp[3]
            markers.scalars[Hr,i] = materials.Hr[3]
            markers.scalars[kThermal,i] = materials.kThermal[3]
        end
    end 
    # end loop over markers
    update_marker_prop!(markers,materials)
end

function get_interface(grid::CartesianGrid,mat::Matrix{Float64},contour_value::Float64)
    # Finding interfaces 
    interface_position = zeros(Float64,grid.nx+1);
    for j in 1:grid.nx+1
        i = 1
        while i <= grid.ny
            if mat[i,j] == contour_value
                interface_position[j] = grid.yc[i]
                break
            elseif mat[i+1,j] < contour_value
                # interface is located within this cell.
                interface_position[j] = grid.yc[i] + (grid.yc[i+1]-grid.yc[i])/(mat[i+1,j]-mat[i,j])*(contour_value-mat[i,j])
                break
            end
            i = i+1
        end
    end
    return interface_position
end
include("Topo.jl")

mk_output_dir (generic function with 1 method)

# Model Setup

In [7]:
options = Dict()
options["wavelength"] = 1e4
options["ice thickness"] = 1e4
options["surface depth"] = 0.50*options["ice thickness"]
options["amplitude"] = 0.10*options["ice thickness"]

function run(options::Dict)
    W = options["wavelength"]
    H = options["ice thickness"] + options["surface depth"] + options["amplitude"] + 1e4
    ny = 251
    nx = Int64(ceil(W/H*ny))
        
    gx = 0.0
    gy = 0.113 

    Tbctype = [-1,-1,1,1] #left, right, top, bottom
#     Tbctype = [1,-1,1,1]
    Tbcval = [0.0,0.0,100.0,273.0] #left, right, top, bottom
#     Tbcval = [273.0,0.0,100.0,273.0]
    bc = BoundaryConditions(0,0,0,0) # currently does nothing but is required argument to stokes solver.
    materials = Materials()

    markx = 6
    marky = 6
    seconds_in_year = 3.15e7
    plot_interval = 1e6*seconds_in_year # plot interval in seconds
    end_time = 3e7*seconds_in_year
    dtmax = plot_interval
    grid = CartesianGrid(W,H,nx,ny)
    println("Creating Markers...")
    @time markers = Markers(grid,["alpha","T","rho","eta","Cp","Hr","kThermal"],["material"] ; nmx=markx,nmy=marky,random=true)
    println("Initial condition...")
    @time initial_conditions!(markers, materials,options)

    local time_plot = []
    local max_topo = []
    local topography = []
    
    ### Setting up agruments for interface function ###
    # initial 
    i_mat, = marker_to_stag(markers,grid,markers.integers[[markers.integerFields["material"]],:],"center")
    i_air_ice_interface = get_interface(grid,i_mat,2.5)
    i_ocean_ice_interface = get_interface(grid,i_mat,1.5)
    initial_max_ice_shell_thickness = maximum(i_ocean_ice_interface-i_air_ice_interface)
    initial_avg_ice_shell_thickness = mean(i_ocean_ice_interface-i_air_ice_interface)
    Ai = initial_max_ice_shell_thickness-initial_avg_ice_shell_thickness

    ### Setting up agruments for termination criteria ###
    max_step::Int64=10
    max_time::Float64=-1.0
    max_time = max_time == -1.0 ? typemax(Float64) : max_time
    max_step = max_step == -1 ? typemax(Int64) : max_step

    time = 0.0
    iout= 0
    last_plot = 0.0
    dt = 1e10

    rho_c = nothing
    rho_vx = nothing 
    rho_vy = nothing 
    alpha = nothing 
    Hr = nothing 
    Cp_c = nothing 
    eta_s = nothing 
    eta_n = nothing 
    vxc = nothing 
    vyc = nothing 
    T = nothing 
    dTmax = nothing 
    dTemp = nothing 
    Tnew = nothing 
    Tlast = nothing 
    x_time = nothing
    kThermal = nothing
    ocean_ice_interface = nothing
    mat = nothing

    itime = 1
    output_dir = "test"

    terminate = false
    while !terminate
        # 0. update the markers properties  
        update_marker_prop!(markers,materials)
        update_marker_temp!(markers,materials)
        # 1. Transfer properties markers -> nodes
        # 1a. Basic Nodes
        eta_s_new, = marker_to_stag(markers,grid,["eta",],"basic")
        # 1b. Cell Centers
        rho_c_new,Cp_c_new,alpha_new,eta_n_new,Tlast_new,Hr_new,kThermal_new = marker_to_stag(markers,grid,["rho","Cp","alpha","eta","T","Hr","kThermal"],"center")
        # 1c. Vx and Vy nodes:
        rho_vx_new, = marker_to_stag(markers,grid,["rho",],"vx")
        rho_vy_new, = marker_to_stag(markers,grid,["rho",],"vy") 

        # deal with any NaN values from interpolation:
        if itime > 1
            if any(isnan.(eta_s_new))
                println("found nan values")
            end
            replace_nan!(eta_s,eta_s_new)
            replace_nan!(rho_c,rho_c_new)
            replace_nan!(Hr,Hr_new)
            replace_nan!(Cp_c,Cp_c_new)
            replace_nan!(alpha,alpha_new)
            replace_nan!(eta_n,eta_n_new)
            replace_nan!(Tlast,Tlast_new)
            replace_nan!(rho_vx,rho_vx_new)
            replace_nan!(rho_vy,rho_vy_new)
            replace_nan!(kThermal,kThermal_new)
        end
        # Copy field data 
        kThermal = copy(kThermal_new)
        rho_vx = copy(rho_vx_new)
        rho_vy = copy(rho_vy_new)
        rho_c = copy(rho_c_new)
        Hr = copy(Hr_new)
        Cp_c = copy(Cp_c_new)
        alpha = copy(alpha_new)
        eta_s = copy(eta_s_new)
        eta_n = copy(eta_n_new)
        Tlast = copy(Tlast_new)

        if itime == 1 
#             println(Tbctype,Tbcval)
            ghost_temperature_center(grid,Tlast,Tbctype,Tbcval)
            cell_center_to_markers!(markers,grid,Tlast,markers.scalars[[markers.scalarFields["T"],],:])
        else
            ghost_temperature_center(grid,Tlast,Tbctype,Tbcval)
        end

        # 2. Assemble and solve the stokes equations
        L,R = form_stokes(grid,eta_s,eta_n,rho_vx,rho_vy,bc,gx,gy;dt=dt)
        stokes_solution = L\R
        vx,vy,P = unpack(stokes_solution,grid;ghost=true)

        # Get the velocity at the cell centers:
        vxc,vyc = velocity_to_centers(grid,vx,vy)
        adiabatic_heating = compute_adiabatic_heating(grid,rho_c,Tlast,alpha,gx,gy,vxc,vyc)*0.0
        shear_heating = compute_shear_heating(grid,vx,vy,eta_n,eta_s)*0.0
        H = (adiabatic_heating .+ shear_heating .+ Hr).*0.0

        # 3. Compute the advection timestep
        if itime > 1
            this_dtmax = min(1.2*dt,dtmax)
        else
            this_dtmax = dtmax
        end
        dt = compute_timestep(grid,vxc,vyc;dtmax=this_dtmax,cfl=0.25)
        diffusion_timestep = (grid.x[2]-grid.x[1])^2 / 1e-6
        if dt > diffusion_timestep
            dt = diffusion_timestep
        end

        dTmax = Inf
        dTemp = nothing
        Tnew = nothing
        titer = 1
        for titer=1:2
            # assemble and solve the energy equation
            println("Trying with timestep ",dt/seconds_in_year/1e3," kyr")
            L,R = assemble_energy_equation_center(grid,rho_c,Cp_c,kThermal,H,Tlast,dt,Tbctype,Tbcval)
            Tnew = L\R;
            Tnew = reshape(Tnew,grid.ny,grid.nx);
            Tnew = ghost_temperature_center(grid,Tnew,Tbctype,Tbcval)
            T = copy(Tnew)

            dTemp = Tnew-Tlast
            # compute the maximum temperature change
            dTmax = maximum(abs.(dTemp[2:end-1,2:end-1]));
            println("dTmax=",dTmax," dt=",dt/seconds_in_year/1e3," kyr")
            dt = min(dt,dTmax < 10.0 ? dt : dt*10.0/dTmax)
            if dTmax < 10.0
                break
            end
        end

        dT_subgrid_node = subgrid_temperature_relaxation_center!(markers,grid,Tlast,Cp_c[1,1],kThermal[1,1],dt)
        dT_remaining = dTemp - dT_subgrid_node

        cell_center_change_to_markers!(markers,grid,dT_remaining,"T")

        # Checking Termination Criteria, time is in Myr
        if time >= max_time || itime >= max_step
            terminate = true
        end
        
        # Checking Termination Criteria, using A/Ai = 1/e
        mat, = marker_to_stag(markers,grid,markers.integers[[markers.integerFields["material"]],:],"center")
        ocean_ice_interface = get_interface(grid,mat,1.5)
        air_ice_interface = get_interface(grid,mat,2.5)
        max_ice_shell_thickness = maximum(ocean_ice_interface.-air_ice_interface)
        avg_ice_shell_thickness = mean(ocean_ice_interface.-air_ice_interface)
        Af = max_ice_shell_thickness-avg_ice_shell_thickness
        if isapprox(Af/Ai,1/exp(1);rtol=0.1)
            # Af/Ai <= 1/exp(1)
            terminate = true
            # println("Finished Step ",itime," time=",time/seconds_in_year/1e3," kyr")
        end   
        
#         if time == 0.0 || mod(itime,100) == 0 || true
#             last_plot = time 
#             # Gird output
#             name1 = @sprintf("%s/viz.%04d.vtr",output_dir,iout)
#             println("Writing visualization file = ",name1)
#             vn = velocity_to_basic_nodes(grid,vxc,vyc)
#             visualization(grid,rho_c,eta_s,vn,P,Tnew[2:end-1,2:end-1],time/seconds_in_year;filename=name1)
#             # Markers output
#             name2 = @sprintf("%s/markers.%04d.vtp",output_dir,iout)
#             println("Writing visualization file = ",name2)
#             visualization(markers,time/seconds_in_year;filename=name2)
#             iout += 1
#         end

        # println("Min/Max velocity: ",minimum(vyc)," ",maximum(vyc))            
        # Moving the markers and advancing to the next timestep
        move_markers_rk4!(markers,grid,vx,vy,dt,continuity_weight=1/3)
        time += dt
        itime += 1
        println("Finished Step ",itime," time=",time/seconds_in_year/1e3," kyr")
        append!(time_plot,time)
        append!(max_topo,maximum(ocean_ice_interface.-(options["ice thickness"] + options["surface depth"])))
        append!(topography,[ocean_ice_interface])
    end
    return markers,grid,i_mat,mat,time_plot,max_topo,topography,max_step,time,itime
end

markers,grid,i_mat,mat,times,max_topo,topography,max_step,time,itime = run(options)

air_ice_interface = get_interface(grid,mat,2.5)
ocean_ice_interface = get_interface(grid,mat,1.5)
i_air_ice_interface = get_interface(grid,i_mat,2.5)
i_ocean_ice_interface = get_interface(grid,i_mat,1.5)


Creating Markers...
  0.018987 seconds (19 allocations: 88.991 MiB)
Initial condition...
  2.414396 seconds (7.28 M allocations: 113.001 MiB, 5.35% gc time)
Trying with timestep 0.34446649029982357 kyr
dTmax=0.9689191143214657 dt=0.34446649029982357 kyr
Finished Step 2 time=0.34446649029982357 kyr
Trying with timestep 0.34446649029982357 kyr
dTmax=0.8966758873171443 dt=0.34446649029982357 kyr
Finished Step 3 time=0.6889329805996471 kyr
Trying with timestep 0.34446649029982357 kyr
dTmax=0.7946231783583926 dt=0.34446649029982357 kyr
Finished Step 4 time=1.0333994708994707 kyr
Trying with timestep 0.34446649029982357 kyr
dTmax=0.7134738400797573 dt=0.34446649029982357 kyr
Finished Step 5 time=1.3778659611992943 kyr
Trying with timestep 0.34446649029982357 kyr
dTmax=0.8029035594105949 dt=0.34446649029982357 kyr
Finished Step 6 time=1.7223324514991178 kyr
Trying with timestep 0.34446649029982357 kyr
dTmax=0.8850688985314719 dt=0.34446649029982357 kyr
Finished Step 7 time=2.0667989417989414 

98-element Vector{Float64}:
 14993.56573141011
 15041.485624056955
 15094.673161556644
 15166.897818934447
 15224.698867170733
 15288.488229999573
 15350.792938515542
 15406.177952962904
 15467.557417911894
 15523.310425853813
 15587.652886552632
 15627.967082633982
 15687.92818852254
     ⋮
 14362.440483904977
 14416.159418085681
 14467.56816426737
 14537.077126808133
 14582.354513565722
 14653.07401433223
 14697.438482093916
 14771.382702212642
 14842.973012623032
 14891.365741706457
 14961.857659326937
 14997.232780681721

In [ ]:
# Profile of ice-water inferface topograpgy over time 
figure()
for i in 1:1:itime-1
    plot(grid.xc,topography[i],label=(L"At",@sprintf("%.3g",times[i]/3.15e7/1e3),L"kyr"))
end
title(L"Profile\,\,of\,\,Ice-Water\,\,Interface\,\,Topography\,\,Over\,\,Time")
#     gca().set_xlim([0.0,1e4])
#     gca().set_ylim([1.8e4,2.2e4])
gca().invert_yaxis()
# Legend is at the bottom
legend(loc="upper center", bbox_to_anchor=(0.5, -0.15),fancybox="True",shadow="True", ncol=5)
show()

In [ ]:
using EasyFit
using PyPlot

ampfit = []
for i in 1:itime-1
    amp = maximum(topography[i])-minimum(topography[i])
    append!(ampfit,amp)
end
x = convert(Array{Float64}, times)
y = convert(Array{Float64}, ampfit)
fit = fitexp(x/3.15e7,y)
figure()
for i in 1:itime-1
    plot(times[i]/3.15e7,maximum(topography[i])-minimum(topography[i]),".") 
end
plot(fit.x,fit.y,"r-",label="fitted data")
gca().set_ylabel(L"Amplitude\,(m)")
gca().set_xlabel(L"Time\,(years)")
legend(fancybox="True",shadow="True")
show()

# Topography info

In [ ]:
println("The maximum total initial thickness of the icy shell is ",@sprintf("%.3g",maximum(i_ocean_ice_interface-i_air_ice_interface)/1000),"(km)")
i_ice_avg = get_ice_thickness(i_air_ice_interface,i_ocean_ice_interface,"initial")
println("The average initial thickness of the icy shell is ",@sprintf("%.3g",i_ice_avg/1000),"(km)")
i_amp = get_amplitude(i_ice_avg,i_air_ice_interface,i_ocean_ice_interface,"initial")
println("The initial amplitude is ",@sprintf("%.3g",i_amp/1000),"(km)")

println("The maximum total thickness of the icy shell after $x_time Myr is ",@sprintf("%.3g",maximum(ocean_ice_interface-air_ice_interface)/1000),"(km)")
ice_avg = get_ice_thickness(air_ice_interface,ocean_ice_interface,"after")
println("The average thickness of the icy shell after $x_time Myr is ",@sprintf("%.3g",ice_avg/1000),"(km)")
amp = get_amplitude(ice_avg,air_ice_interface,ocean_ice_interface,"after")
print("The amplitude after $x_time Myr is ",@sprintf("%.3g",amp/1000),"(km)")

# Initial Plot Profiles

In [ ]:
# Inital Model Schematic Profile
figure() 
pcolor(grid.xc,grid.yc,i_mat)
colorbar()
plot(grid.xc,i_air_ice_interface,"k",label="air-ice interface")
plot(grid.xc,i_ocean_ice_interface,"r",label="ocean-ice interface")
title(L"Initial\,\,Model\,\,Schematic")
gca().invert_yaxis()
gca().set_aspect("equal")
gca().set_ylabel(L"Height\,(m)")
gca().set_xlabel(L"Width\,(m)")
legend(loc="upper right",bbox_to_anchor=[1.8,1.0])
show()

# Initial ice shell thickness profile along the horizontal direction
figure() 
plot(grid.xc,(i_ocean_ice_interface-i_air_ice_interface)/1000)
title(L"Initial\,\,Ice\,\,Shell\,\,Thickness\,\,Across\,\,the\,\,Horizontal\,\,Direction")
gca().set_ylabel(L"thickness\,(m)")
gca().set_xlabel(L"x\,(m)")
show()

# After Plot Profiles 

In [ ]:
# Model Schematic Profile
figure()
pcolor(grid.xc/1000,grid.yc/1000,mat)
colorbar()
plot(grid.xc/1000,air_ice_interface/1000,"m",label="air-ice interface")
plot(grid.xc/1000,ocean_ice_interface/1000,"r",label="ocean-ice interface")
title(L"Model\,\,Schematic\,\,at\,\,%$x_time\,Myr")
gca().invert_yaxis()
gca().set_aspect("equal")
gca().set_ylabel(L"Height\,(km)")
gca().set_xlabel(L"Width\,(km)")
legend(loc="upper right",bbox_to_anchor=[1.8,1.0])
show()

# Ice shell thickness profile along the horizontal direction
figure()
plot(grid.xc/1000,(ocean_ice_interface-air_ice_interface)/1000)
title(L"Ice\,\,Shell\,\,Thickness\,\,Across\,\,the\,\,Horizontal\,\,Direction\,\,at\,\,%$x_time\,Myr")
gca().set_ylabel(L"thickness\,(km)")
gca().set_xlabel(L"x\,(km)")
show()

# Profile of maximum topograpgy over time 
figure()
plot(time_plot/seconds_in_year/1e3,max_topo/1000)
title(L"Maximum\,\,Topography\,\,Over\,\,Time")
gca().set_ylabel(L"Max.\,topography\,(km)")
gca().set_xlabel(L"Time\,(ka)")
show()

# Temperature Profile 
figure()
scatter(markers.x[1,:]/1000,markers.x[2,:]/1000,c=markers.scalars[markers.scalarFields["T"],:],s=0.1)
plot(grid.xc/1000,air_ice_interface/1000,"m",label="air-ice interface")
plot(grid.xc/1000,ocean_ice_interface/1000,"r",label="ocean-ice interface")
title(L"Temperature\,\,Profile\,\,at\,\,%$x_time\,Myr")
colorbar(label=L"K")
gca().invert_yaxis()
gca().set_aspect("equal")
gca().set_ylabel(L"Height\,(km)")
gca().set_xlabel(L"Width\,(km)")
legend(loc="upper right",bbox_to_anchor=[1.8,1.0])
show()

# Viscosity Profile 
figure()
scatter(markers.x[1,:]/1000,markers.x[2,:]/1000,c=log10.(markers.scalars[markers.scalarFields["eta"],:]),s=0.1)
plot(grid.xc/1000,air_ice_interface/1000,"m",label="air-ice interface")
plot(grid.xc/1000,ocean_ice_interface/1000,"r",label="ocean-ice interface")
title(L"Viscosity\,\,Profile\,\,at\,\,%$x_time\,Myr")
colorbar(label=L"Pa\cdot{s}")
gca().invert_yaxis()
gca().set_aspect("equal")
gca().set_ylabel(L"Height\,(km)")
gca().set_xlabel(L"Width\,(km)")
legend(loc="upper right",bbox_to_anchor=[1.8,1.0])
show()

In [ ]:
function time_viscous(lambda::Float64)
    """
    Arguments:
    lambda::Float64 --- Topography Wavelength (m)
    
    Returns:
    time --- Time for viscous flow (yrs)
    """
    g = 0.113
    ice_viscosity = 10^14
    delta_rho = 80
    time = (4*pi)*(ice_viscosity)*(1/g)*(1/delta_rho)*(1/lambda)
    return time/3.15e7
end

function thickening_rate(hice::Float64)
    """
    Arguments:
    hice::Float64 --- Ice shell thickness (m)
    
    Returns:
    rate --- Rate of thickening, units are (m/s)

    Info:
    1W = 1J/s
    q = -k*(T-To)/h
    """
    delta_T = 173 # Temperature (K)
    kthermal = 2.2 # Thermal conductivity (W/m*K)
    L = 334*10^3 # Latent heat of fusion (J/kg)
    rho = 1000 # Density (kg/m^3)
    q = kthermal*(delta_T/hice)
    rate = q/L/rho   
    return rate
end

function thickening_time(hice::Float64)
    """
    Arguments:
    hice::Float64 --- Ice shell thickness (m)
    
    Returns:
    time --- Time for thickening (yrs)

    Info:
    1W = 1J/s
    q = -k*(T-To)/h
    """
    rate = thickening_rate(hice)
    time = hice/rate
    return time/3.15e7
end